## Day 4 — SAT Results ETL

## 1. Data Exporation

In [1]:
import os
import pandas as pd

# Get the current working directory (where your Jupyter notebook is saved)
cwd = os.getcwd()

# Build the file path relative to your notebook
file_path = os.path.join(cwd, "sat-results.csv")

# Load the CSV
df = pd.read_csv(file_path)

print("File loaded successfully:", df.shape)
df.head()

File loaded successfully: (493, 11)


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0


## 2. Cleaning 
- Drop duplicate column `SAT Critical Readng Avg. Score`
- Convert numeric columns from strings to integers
- Strip `%` from `pct_students_tested` and convert to float
- Remove outliers (SAT scores must be between 200–800)
- Drop unrelated columns (`internal_school_id`, `contact_extension`)
- Standardize column headers to snake_case

In [3]:
# Example: clean numeric columns
df = df.rename(columns={
    "SCHOOL NAME": "school_name",
    "Num of SAT Test Takers": "num_test_takers",
    "SAT Critical Reading Avg. Score": "sat_reading_avg",
    "SAT Math Avg. Score": "sat_math_avg",
    "SAT Writing Avg. Score": "sat_writing_avg",
    "pct_students_tested": "pct_students_tested",
    "academic_tier_rating": "academic_tier_rating"
})

# Drop duplicate/dirty columns
df = df.drop(columns=["SAT Critical Readng Avg. Score", "internal_school_id", "contact_extension"])

# Convert datatypes
numeric_cols = ["num_test_takers", "sat_reading_avg", "sat_math_avg", "sat_writing_avg"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Convert pct_students_tested
df["pct_students_tested"] = df["pct_students_tested"].str.replace("%", "", regex=False)
df["pct_students_tested"] = pd.to_numeric(df["pct_students_tested"], errors="coerce")

# Remove outliers
for col in ["sat_reading_avg", "sat_math_avg", "sat_writing_avg"]:
    df = df[(df[col] >= 200) & (df[col] <= 800)]

df.head()
    

,DBN,school_name,num_test_takers,sat_reading_avg,sat_math_avg,sat_writing_avg,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29.0,355.0,404.0,363.0,78.0,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91.0,383.0,423.0,366.0,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70.0,377.0,402.0,370.0,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7.0,414.0,401.0,359.0,92.0,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44.0,390.0,433.0,384.0,92.0,2.0


## 3. Quality checks (missing values, basic stats)

In [19]:
print("Row count:", len(df))
print("\nMissing values per column:")
print(df.isna().sum())

print("\nDescriptive statistics (numeric):")
display(df.describe())

print("\nSample rows:")
display(df.sample(5, random_state=42))

Row count: 493

Missing values per column:
dbn                       0
school_name               0
num_test_takers          58
sat_reading_avg          58
sat_math_avg             58
sat_writing_avg          58
pct_students_tested     117
academic_tier_rating     91
dtype: int64

Descriptive statistics (numeric):


,num_test_takers,sat_reading_avg,sat_math_avg,sat_writing_avg,pct_students_tested,academic_tier_rating
count,435.000000,435.000000,435.000000,435.000000,376.000000,402.000000
mean,111.402299,400.712644,417.643678,393.712644,84.683511,2.564677
std,154.806216,56.056774,86.991283,57.870447,5.718120,1.126443
min,6.000000,279.000000,-10.000000,286.000000,78.000000,1.000000
25%,42.000000,368.000000,372.000000,360.000000,78.000000,2.000000
50%,63.000000,391.000000,396.000000,382.000000,85.000000,3.000000
75%,97.500000,416.000000,438.000000,411.000000,92.000000,4.000000
max,1277.000000,679.000000,1100.000000,682.000000,92.000000,4.000000



Sample rows:


,dbn,school_name,num_test_takers,sat_reading_avg,sat_math_avg,sat_writing_avg,pct_students_tested,academic_tier_rating
454,31R450,CURTIS HIGH SCHOOL,375.0,437.0,435.0,429.0,92.0,4.0
73,02M655,LIFE SCIENCES SECONDARY SCHOOL,61.0,409.0,424.0,403.0,92.0,2.0
310,17K548,BROOKLYN SCHOOL FOR MUSIC & THEATRE,48.0,385.0,393.0,373.0,78.0,1.0
175,09X505,"BRONX SCHOOL FOR LAW, GOVERNMENT AND JUSTICE",80.0,404.0,418.0,402.0,NaN,4.0
332,19K431,THOMAS JEFFERSON YABC,NaN,NaN,NaN,NaN,92.0,2.0


In [7]:
import pandas as pd

# === 1. Load raw SAT results ===
df = pd.read_csv("sat-results.csv")

print("Original shape:", df.shape)
print("Original columns:", df.columns.tolist())
print(df.head(3))


# === 2. Normalize headers ===
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)

print("\nNormalized columns:", df.columns.tolist())


# === 3. Remove % signs, convert to numeric ===
for col in df.columns:
    if df[col].dtype == "object":
        # Try cleaning percentages
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("%", "", regex=False)
            .str.replace(",", "", regex=False)
        )

        # Convert to numeric where possible
        df[col] = pd.to_numeric(df[col], errors="ignore")

# === 4. Drop duplicates ===
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"\nDropped {before - after} duplicate rows.")


# === 5. Handle SAT score columns (200–800 valid range) ===
sat_cols = [c for c in df.columns if "sat" in c]

for col in sat_cols:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].where((df[col] >= 200) & (df[col] <= 800))


# === 6. Inspect cleaned data ===
print("\nCleaned data preview:")
print(df.head(5))
print("\nSAT score stats:")
print(df[sat_cols].describe())


# === 7. Save cleaned version ===
df.to_csv("cleaned_sat_results.csv", index=False)
print("\n✅ Cleaned dataset saved as 'cleaned_sat_results.csv'")

Original shape: (493, 11)
Original columns: ['DBN', 'SCHOOL NAME', 'Num of SAT Test Takers', 'SAT Critical Reading Avg. Score', 'SAT Math Avg. Score', 'SAT Writing Avg. Score', 'SAT Critical Readng Avg. Score', 'internal_school_id', 'contact_extension', 'pct_students_tested', 'academic_tier_rating']
      DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   

  Num of SAT Test Takers SAT Critical Reading Avg. Score SAT Math Avg. Score  \
0                     29                             355                 404   
1                     91                             383                 423   
2                     70                             377                 402   

  SAT Writing Avg. Score SAT Critical Readng Avg. Score  internal_school_id  \
0                    363                            355   

/var/folders/hc/7hm0sy6n535fnm_9p0crxbmw0000gn/T/ipykernel_22164/2539915868.py:35: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/hc/7hm0sy6n535fnm_9p0crxbmw0000gn/T/ipykernel_22164/2539915868.py:35: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/hc/7hm0sy6n535fnm_9p0crxbmw0000gn/T/ipykernel_22164/2539915868.py:35: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/hc/7hm0sy6n535fnm_9p0crxbmw0000gn/T/ipykernel_22164/2539915868.py:35: FutureWarning: errors='ignore

In [8]:
# === 3. Remove % signs, commas, and convert to numeric where possible ===
for col in df.columns:
    if df[col].dtype == "object":
        # Clean percentages and commas
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("%", "", regex=False)
            .str.replace(",", "", regex=False)
        )

        # Try converting to numeric
        try:
            df[col] = pd.to_numeric(df[col])
        except ValueError:
            # If conversion fails, keep as text
            pass

In [9]:
# Identify SAT score columns (adjust names if needed)
sat_columns = ["sat_math_avg_score", "sat_critical_reading_avg_score", "sat_writing_avg_score"]

for col in sat_columns:
    if col in df.columns:
        # Remove % or commas if present
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("%", "", regex=False)
            .str.replace(",", "", regex=False)
        )
        
        # Convert to numeric
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [10]:
import pandas as pd

# Load dataset (update path as needed)
df = pd.read_csv("sat-results.csv")

# Columns for SAT scores
sat_columns = [
    "sat_math_avg_score",
    "sat_critical_reading_avg_score",
    "sat_writing_avg_score"
]

# Clean SAT columns
for col in sat_columns:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("%", "", regex=False)
            .str.replace(",", "", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

        # Keep only valid SAT scores (200–800)
        df.loc[(df[col] < 200) | (df[col] > 800), col] = None

# Drop rows wit

In [11]:
import pandas as pd

# Load your dataset
df = pd.read_csv("sat-results.csv")

# Correct SAT score columns based on your file
sat_columns = [
    "SAT Critical Reading Avg. Score",
    "SAT Math Avg. Score",
    "SAT Writing Avg. Score"
]

# Clean SAT columns
for col in sat_columns:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("%", "", regex=False)   # remove % if any
            .str.replace(",", "", regex=False)   # remove commas
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

        # Keep only valid SAT scores (200–800)
        df.loc[(df[col] < 200) | (df[col] > 800), col] = None

# Drop rows where all SAT scores are missing
df_clean = df.dropna(subset=sat_columns, how="all")

print("✅ Data cleaned. Shape:", df_clean.shape)
df_clean.head()

✅ Data cleaned. Shape: (435, 11)


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,390,672714,x123,92%,2.0


## CHECK COLUMN NAMES

In [12]:
# Normalize column names
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^a-z0-9_]", "", regex=True)  # remove non-alphanumeric chars
)

# Check new column names
print(df.columns.tolist())

['dbn', 'school_name', 'num_of_sat_test_takers', 'sat_critical_reading_avg_score', 'sat_math_avg_score', 'sat_writing_avg_score', 'sat_critical_readng_avg_score', 'internal_school_id', 'contact_extension', 'pct_students_tested', 'academic_tier_rating']


In [13]:
sat_columns = [
    "sat_critical_reading_avg_score",
    "sat_math_avg_score",
    "sat_writing_avg_score"
]

for col in sat_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df.loc[(df[col] < 200) | (df[col] > 800), col] = None

# Drop rows where all SAT scores are missing
df_clean = df.dropna(subset=sat_columns, how="all")

print("✅ Data cleaned. Shape:", df_clean.shape)
df_clean.head()

✅ Data cleaned. Shape: (435, 11)


,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,sat_critical_readng_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,390,672714,x123,92%,2.0


In [14]:
import pandas as pd

# Assuming df_clean is ready from the cleaning step
# Add a total SAT score column
df_clean["total_sat"] = (
    df_clean["sat_critical_reading_avg_score"] +
    df_clean["sat_math_avg_score"] +
    df_clean["sat_writing_avg_score"]
)

# 1. Schools with valid SAT scores
valid_schools = df_clean["dbn"].nunique()
print("✅ Schools with valid SAT scores:", valid_schools)

# 2. Average SAT scores
avg_scores = df_clean[["sat_critical_reading_avg_score",
                       "sat_math_avg_score",
                       "sat_writing_avg_score",
                       "total_sat"]].mean().round(2)
print("\n📊 Average SAT Scores:")
print(avg_scores)

# 3. Top 10 schools by total SAT
top10 = df_clean.nlargest(10, "total_sat")[["school_name", "total_sat"]]
print("\n🏆 Top 10 Schools by Total SAT:")
print(top10)

# 4. Worst 10 schools by total SAT
bottom10 = df_clean.nsmallest(10, "total_sat")[["school_name", "total_sat"]]
print("\n⚠️ Bottom 10 Schools by Total SAT:")
print(bottom10)

# 5. Distribution summary for Math
print("\n📈 Math Score Distribution:")
print(df_clean["sat_math_avg_score"].describe())

✅ Schools with valid SAT scores: 421

📊 Average SAT Scores:
sat_critical_reading_avg_score     400.71
sat_math_avg_score                 413.34
sat_writing_avg_score              393.71
total_sat                         1208.16
dtype: float64

🏆 Top 10 Schools by Total SAT:
                                           school_name  total_sat
48                              STUYVESANT HIGH SCHOOL     2096.0
198                       BRONX HIGH SCHOOL OF SCIENCE     1969.0
459                STATEN ISLAND TECHNICAL HIGH SCHOOL     1953.0
206  HIGH SCHOOL OF AMERICAN STUDIES AT LEHMAN COLLEGE     1920.0
396                        TOWNSEND HARRIS HIGH SCHOOL     1910.0
427  QUEENS HIGH SCHOOL FOR THE SCIENCES AT YORK CO...     1868.0
8                       BARD HIGH SCHOOL EARLY COLLEGE     1856.0
107  HIGH SCHOOL FOR MATHEMATICS, SCIENCE AND ENGIN...     1847.0
249                     BROOKLYN TECHNICAL HIGH SCHOOL     1833.0
33                       ELEANOR ROOSEVELT HIGH SCHOOL     1758.0

/var/folders/hc/7hm0sy6n535fnm_9p0crxbmw0000gn/T/ipykernel_22164/2754375111.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["total_sat"] = (


In [15]:
import pandas as pd
df=pd.read_csv('sat-results.csv')
df

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0
...,...,...,...,...,...,...,...,...,...,...,...
488,27Q480,JOHN ADAMS HIGH SCHOOL,403,391,409,392,391,863765,NaN,92%,1.0
489,13K605,GEORGE WESTINGHOUSE CAREER AND TECHNICAL EDUCA...,85,406,391,392,406,937579,x234,NaN,NaN
490,05M304,MOTT HALL HIGH SCHOOL,54,413,399,398,413,296405,x123,78%,2.0
491,02M520,MURRY BERGTRAUM HIGH SCHOOL FOR BUSINESS CAREERS,264,407,440,393,407,892839,NaN,92%,2.0


In [16]:
new_df=df.drop_duplicates()
new_df

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0
...,...,...,...,...,...,...,...,...,...,...,...
473,75X012,P.S. X012 LEWIS AND CLARK SCHOOL,s,s,s,s,s,733698,x234,92%,1.0
474,75X754,J. M. RAPPORT SCHOOL CAREER DEVELOPMENT,s,s,s,s,s,976034,x345,NaN,4.0
475,79M645,SCHOOL FOR COOPERATIVE TECHNICAL EDUCATION,s,s,s,s,s,556924,x123,85%,1.0
476,79Q950,GED PLUS s CITYWIDE,8,496,400,426,496,544514,x234,92%,2.0
